In [43]:
import SimpleITK as sitk

def multires_registration(fixed_image, moving_image, initial_transform):    
    registration_method = sitk.ImageRegistrationMethod()
    #registration_method.SetInterpolator(sitk.sitkLinear)
    registration_method.SetInterpolator(sitk.sitkBSpline)
    #registration_method.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
    registration_method.SetMetricAsMeanSquares()
    registration_method.SetMetricSamplingStrategy(registration_method.RANDOM)
    registration_method.SetMetricSamplingPercentage(0.01)
    registration_method.SetOptimizerAsGradientDescent(learningRate=1.0, numberOfIterations=100, convergenceMinimumValue=1e-6, convergenceWindowSize=55) # Reduce convergeMinimumValue=1e-6
    #increase convergenceWindowsize=10. http://insightsoftwareconsortium.github.io/SimpleITK-Notebooks/Python_html/60_Registration_Introduction.html 
    registration_method.SetOptimizerScalesFromPhysicalShift() 
    registration_method.SetInitialTransform(initial_transform, inPlace=True)
    registration_method.SetShrinkFactorsPerLevel(shrinkFactors = [4,2,1])
    registration_method.SetSmoothingSigmasPerLevel(smoothingSigmas = [2,1,0])
    registration_method.SmoothingSigmasAreSpecifiedInPhysicalUnitsOn()

    registration_method.Execute(fixed_image, moving_image)

    print('Final metric value: {0}'.format(registration_method.GetMetricValue()))
    print('Optimizer\'s stopping condition, {0}'.format(registration_method.GetOptimizerStopConditionDescription()))

In [74]:
# Read the images
fixed_image = sitk.ReadImage('1.PNG', sitk.sitkFloat32)
moving_image = sitk.ReadImage('2.PNG', sitk.sitkFloat32)

# Display the original images and resamples moving_image (onto the
# fixed_image grid using the identity transformation)
sitk.Show(fixed_image, 'fixed')
sitk.Show(moving_image, 'moving')
#sitk.Show(sitk.Resample(moving_image, fixed_image, sitk.Transform()), 'identity transform')

# Centered 2D affine transform and show the resampled moving_image using this transform.
registration_transform = sitk.CenteredTransformInitializer(fixed_image, 
                                                      moving_image, 
                                                      sitk.AffineTransform(2), 
                                                      sitk.CenteredTransformInitializerFilter.GEOMETRY)
#sitk.Show(sitk.Resample(moving_image, fixed_image, registration_transform), 'initial affine transform')


# Register using 2D affine initial transform that is overwritten
# and show the resampled moving_image using this transform.
multires_registration(fixed_image, moving_image, registration_transform)
sitk.Show(sitk.Resample(moving_image, fixed_image, registration_transform), 'final affine transform')
registered_image = sitk.Resample(moving_image, fixed_image, registration_transform)

Final metric value: 1237.6245114436922
Optimizer's stopping condition, GradientDescentOptimizerv4Template: Maximum number of iterations (100) exceeded.


In [76]:
# http://insightsoftwareconsortium.github.io/SimpleITK-Notebooks/Python_html/05_Results_Visualization.html
zeros = sitk.Image(fixed_image.GetSize(), fixed_image.GetPixelID())
zeros.CopyInformation(fixed_image)

#sitk.Cast(sitk.Compose(fixed_image, moving_image, zeros), sitk.sitkVectorUInt8)
#sitk.Cast(sitk.Compose(fixed_image, moving_image, fixed_image), sitk.sitkVectorUInt8)
#sitk.Cast(sitk.Compose(fixed_image, 0.5*fixed_image+0.5*moving_image, moving_image), sitk.sitkVectorUInt8)]

sitk.Show(sitk.Cast(sitk.Compose(fixed_image, 0.5*fixed_image+0.5*registered_image, registered_image), sitk.sitkVectorUInt8), "fixed and moved")
sitk.Show(sitk.Cast(sitk.Compose(fixed_image, 0.5*fixed_image+0.5*moving_image, moving_image), sitk.sitkVectorUInt8), "fixed and registered")